# Routes

> FastHTML route handlers for settings interface

In [ ]:
#| default_exp routes

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
import json
from pathlib import Path
from typing import Dict, Any, Optional, Callable
from fasthtml.common import *
from fasthtml.common import FT

from cjm_fasthtml_app_core.components.alerts import create_error_alert, create_success_alert
from cjm_fasthtml_settings.core.html_ids import SettingsHtmlIds as HtmlIds
from cjm_fasthtml_settings.core.config import DEFAULT_CONFIG_DIR
from cjm_fasthtml_settings.core.schemas import registry
from cjm_fasthtml_settings.core.utils import (
    save_config,
    get_default_values_from_schema,
    convert_form_data_to_config,
)
from cjm_fasthtml_settings.components.forms import create_settings_form_container

In [ ]:
#| export
# Optional: Import error handling library if available
try:
    from cjm_error_handling.core.base import ErrorContext, ErrorSeverity
    from cjm_error_handling.core.errors import ConfigurationError, ValidationError
    _has_error_handling = True
except ImportError:
    _has_error_handling = False

## Configuration

Module-level configuration that can be customized by users before importing the router.

In [ ]:
#| export
# Configuration settings that users can override
class RoutesConfig:
    """Configuration for settings routes behavior."""
    config_dir: Path = DEFAULT_CONFIG_DIR
    default_schema: str = "general"
    menu_section_title: str = "Settings"
    wrap_with_layout: Optional[Callable] = None  # Optional function to wrap full page content
    plugin_registry: Optional[Any] = None  # Optional plugin registry (must implement PluginRegistryProtocol)

# Module-level config instance
config = RoutesConfig()

### Configuring Settings

The `configure_settings()` function is a convenience helper that sets all configuration options at once, providing a cleaner alternative to setting `config` attributes individually.

This is the recommended way to configure the settings system before adding the router to your app.

In [ ]:
#| export
def configure_settings(
    config_dir: Path = None,  # Directory for storing configuration files
    wrap_with_layout: Callable = None,  # Function to wrap full page content with app layout
    plugin_registry = None,  # Optional plugin registry (must implement PluginRegistryProtocol)
    default_schema: str = "general",  # Default schema to display
    menu_section_title: str = "Settings"  # Title for the settings menu section
) -> RoutesConfig:  # Configured RoutesConfig instance
    """Configure the settings system with a single function call."""
    if config_dir is not None:
        config.config_dir = config_dir
    if wrap_with_layout is not None:
        config.wrap_with_layout = wrap_with_layout
    if plugin_registry is not None:
        config.plugin_registry = plugin_registry
    if default_schema is not None:
        config.default_schema = default_schema
    if menu_section_title is not None:
        config.menu_section_title = menu_section_title
    
    return config

## Error Handling Integration

When the `cjm-error-handling` library is installed, settings routes use structured errors for better error tracking:

- **`ConfigurationError`**: Raised for config load/save failures, missing config files, or parse errors
- **`ValidationError`**: Raised for invalid configuration data or schema validation failures

These structured errors provide:
- User-friendly messages for alerts
- Debug information for logging
- Rich context (schema_id, operation, config_path, etc.)
- Error serialization for API responses

**Without the library**: Falls back to returning error alert components directly

**Usage**:
```python
try:
    if save_config(schema_id, config_data, config_dir):
        # Success
        pass
except ConfigurationError as e:
    # Handle structured error
    return create_error_alert(e.get_user_message())
```

## Helper Functions

In [ ]:
#| export
def _resolve_schema(
    id: str  # Schema ID
) -> tuple:  # (schema, error_message)
    """Resolve schema from ID using the registry."""
    return registry.resolve_schema(id)

## API Router and Routes

The module-level `settings_ar` router can be directly imported and attached to your FastHTML app.

In [ ]:
#| export
# Module-level API router
settings_ar = APIRouter(prefix="/settings")

In [ ]:
#| export
@settings_ar
def index(
    request,  # FastHTML request object
    id: str = None  # Schema ID to display (defaults to config.default_schema)
) -> FT:  # Settings page content
    """Main settings page."""
    from cjm_fasthtml_settings.components.master_detail_adapter import create_settings_master_detail
    from cjm_fasthtml_app_core.core.htmx import is_htmx_request
    from cjm_fasthtml_interactions.core.html_ids import InteractionHtmlIds
    
    if id is None:
        id = config.default_schema
        
    schema, _ = _resolve_schema(id)
    
    # Create the master-detail instance
    settings_md = create_settings_master_detail(
        schemas=registry.get_all(),
        config_dir=config.config_dir,
        save_route_fn=lambda schema_id: save.to(id=schema_id),
        reset_route_fn=lambda schema_id: reset.to(id=schema_id),
        default_schema=config.default_schema,
        menu_section_title=config.menu_section_title,
        plugin_registry=config.plugin_registry,
        plugin_save_route_fn=lambda plugin_id: plugin_save.to(id=plugin_id),
        plugin_reset_route_fn=lambda plugin_id: plugin_reset.to(id=plugin_id)
    )
    
    # For HTMX requests targeting the detail area specifically, return just the detail content
    # This happens when clicking between settings items within the interface
    hx_target = request.headers.get('HX-Target')
    if is_htmx_request(request) and hx_target == InteractionHtmlIds.MASTER_DETAIL_DETAIL:
        item = settings_md.get_item(id)
        if item:
            ctx = settings_md.create_context(request, request.session, item)
            content = settings_md.render_detail(item, ctx)
            
            # Add OOB sidebar update
            master_oob = settings_md.render_master_oob(
                active_item_id=id,
                item_route_func=lambda iid: index.to(id=iid)
            )
            
            return Div(content, master_oob)
    
    # For full page requests or HTMX requests from outside (e.g., navbar),
    # render the complete interface
    full_interface = settings_md.render_full_interface(
        active_item_id=id,
        item_route_func=lambda iid: index.to(id=iid),
        request=request,
        sess=request.session
    )
    
    # Wrap with layout if provided and not an HTMX request
    if config.wrap_with_layout and not is_htmx_request(request):
        return config.wrap_with_layout(full_interface)
    return full_interface

In [ ]:
#| export
@settings_ar
async def save(
    request,  # FastHTML request object
    id: str  # Schema ID to save
) -> FT:  # Response with form or error
    """Save configuration handler."""
    from cjm_fasthtml_interactions.core.html_ids import InteractionHtmlIds
    
    schema, error_msg = _resolve_schema(id)
    if error_msg:
        return create_error_alert(error_msg)
    
    form_data = await request.form()
    config_data = convert_form_data_to_config(form_data, schema)
    
    # Save configuration
    if save_config(id, config_data, config.config_dir):
        alert_msg = create_success_alert(f"Configuration saved for {schema.get('title')}")
        return create_settings_form_container(
            schema=schema,
            values=config_data,
            post_url=save.to(id=id),
            reset_url=reset.to(id=id),
            alert_message=alert_msg,
            target_id=InteractionHtmlIds.MASTER_DETAIL_DETAIL
        )
    else:
        return create_error_alert(f"Failed to save {schema.get('title')} configuration")

In [ ]:
#| export
@settings_ar
def reset(
    id: str  # Schema ID to reset
) -> FT:  # Response with form or error
    """Reset configuration to defaults handler."""
    from cjm_fasthtml_interactions.core.html_ids import InteractionHtmlIds
    
    schema, error_msg = _resolve_schema(id)
    if error_msg:
        return create_error_alert(error_msg)
    
    # Use only default values
    values = get_default_values_from_schema(schema)
    alert_msg = create_success_alert("Configuration reset to defaults")
    
    return create_settings_form_container(
        schema=schema,
        values=values,
        post_url=save.to(id=id),
        reset_url=reset.to(id=id),
        alert_message=alert_msg,
        target_id=InteractionHtmlIds.MASTER_DETAIL_DETAIL
    )

## Plugin Routes (Optional)

These routes are only active if `config.plugin_registry` is set.

In [ ]:
#| export
@settings_ar
def plugin_reset(
    id: str  # Plugin unique ID
) -> FT:  # Response with form or error
    """Reset plugin configuration to defaults handler."""
    from cjm_fasthtml_interactions.core.html_ids import InteractionHtmlIds
    
    if not config.plugin_registry:
        return create_error_alert("Plugin system not configured")
    
    plugin_metadata = config.plugin_registry.get_plugin(id)
    if not plugin_metadata:
        return create_error_alert("Plugin not found")
    
    schema = plugin_metadata.config_schema
    values = get_default_values_from_schema(schema)
    alert_msg = create_success_alert("Configuration reset to defaults")
    
    return create_settings_form_container(
        schema=schema,
        values=values,
        post_url=plugin_save.to(id=id),
        reset_url=plugin_reset.to(id=id),
        alert_message=alert_msg,
        target_id=InteractionHtmlIds.MASTER_DETAIL_DETAIL
    )

In [ ]:
#| export
@settings_ar
async def plugin_save(
    request,  # FastHTML request object
    id: str  # Plugin unique ID
) -> FT:  # Response with form or error
    """Save plugin configuration handler."""
    from cjm_fasthtml_interactions.core.html_ids import InteractionHtmlIds
    
    if not config.plugin_registry:
        return create_error_alert("Plugin system not configured")
    
    plugin_metadata = config.plugin_registry.get_plugin(id)
    if not plugin_metadata:
        return create_error_alert("Plugin not found")
    
    form_data = await request.form()
    schema = plugin_metadata.config_schema
    config_data = convert_form_data_to_config(form_data, schema)
    
    # Save configuration
    if config.plugin_registry.save_plugin_config(id, config_data):
        alert_msg = create_success_alert(f"Configuration saved for {plugin_metadata.title}")
        return create_settings_form_container(
            schema=schema,
            values=config_data,
            post_url=plugin_save.to(id=id),
            reset_url=plugin_reset.to(id=id),
            alert_message=alert_msg,
            target_id=InteractionHtmlIds.MASTER_DETAIL_DETAIL
        )
    else:
        return create_error_alert("Failed to save configuration")

## Usage Example

In [ ]:
# Example: Using the settings router in your app
from cjm_fasthtml_settings.core.schemas import registry
from cjm_fasthtml_settings.core.config import get_app_config_schema

# First, register your schemas
registry.register(get_app_config_schema(app_title="My App", include_theme=False))
registry.register({
    "name": "database",
    "title": "Database Settings",
    "type": "object",
    "properties": {
        "host": {"type": "string", "title": "Host", "default": "localhost"}
    }
})

# NEW WAY: Use the configure_settings helper (recommended)
from cjm_fasthtml_settings.routes import configure_settings
from pathlib import Path

configure_settings(
    config_dir=Path("my_configs"),
    default_schema="general",
    menu_section_title="Settings"
)

# OLD WAY: Direct config attribute setting (still works)
# from cjm_fasthtml_settings.routes import config
# config.config_dir = Path("my_configs")
# config.default_schema = "general"

# Then import and use the router
from cjm_fasthtml_settings.routes import settings_ar, config

print(f"Settings router prefix: {settings_ar.prefix}")
print(f"Registered schemas: {registry.list_schemas()}")
print(f"Config directory: {config.config_dir}")

Settings router prefix: /settings
Registered schemas: ['general', 'database']
Config directory: my_configs


In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()